In [1]:
import eli5
import pandas as pd
import re
import numpy as np
import warnings

warnings.filterwarnings("ignore")

from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.linear_model import LinearRegression, LogisticRegression
from sklearn.metrics import f1_score, mean_squared_error, classification_report
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import MaxAbsScaler
from sklearn.feature_extraction.text import CountVectorizer

In [2]:
df = pd.read_csv('/Users/m.e.zubkova/Documents/diploma_final.csv')

In [101]:
df.to_csv('diploma_final.csv', index=False)

In [3]:
male_names = pd.read_json('/Users/m.e.zubkova/Downloads/nlp-gender/male_names.json')
female_names = pd.read_json('/Users/m.e.zubkova/Downloads/nlp-gender/female_names.json')

In [4]:
def sex_classify(name: str):
    if name in male_names.values:
        return 'male'
    if name in female_names.values:
        return 'female'
    return 'not in list of names'

In [5]:
df['freelancer_first_name'] = df.freelancer_name.apply(lambda name: name.split()[0])

In [6]:
df['freelancer_gender'] = df.freelancer_first_name.apply(sex_classify)

Часть имен мы вручную вписали в словарь для определения пола большего числа фрилансеров.

In [7]:
# определим гендер для случаев, когда пользователи поменяли местами имя и фамилию

df.loc[
    (df['freelancer_gender'] == 'not in list of names'), 'freelancer_gender'
] = df.freelancer_name.apply(
    lambda name: sex_classify(name.split()[1]) if len(name.split()) > 1 else 'not in list of names')

In [8]:
df[df['freelancer_gender'] == 'not in list of names'].shape

# без гендера осталось 604 отзыва, их мы не будем учитывать в последующем анализе

(604, 7)

In [9]:
gendered = df[df['freelancer_gender'] != 'not in list of names']
gendered.freelancer_gender.value_counts()

female    6444
male      3781
Name: freelancer_gender, dtype: int64

In [10]:
gendered[gendered.texts == 'Нет отзыва'].freelancer_gender.value_counts()

female    911
male      334
Name: freelancer_gender, dtype: int64

In [11]:
def no_text(texts: pd.Series):
    """
    Функция подсчитывает долю отзывов без текста от всех отзывов (необходимо наличие оценки)
    """
    return len(texts[texts == 'Нет отзыва']) / len(texts)

In [12]:
gendered.groupby('freelancer_gender').agg({'texts': no_text})

# доля отзывов без текста у мужчин ниже, воспользуемся критерием хи-квадрат для проверки

,texts
freelancer_gender,
female,0.141372
male,0.088336


In [13]:
gendered['texts_existence'] = gendered.texts.apply(lambda text: text != 'Нет отзыва')
ct1 = pd.crosstab(gendered['freelancer_gender'], gendered['texts_existence'])
ct1

texts_existence,False,True
freelancer_gender,,
female,911,5533
male,334,3447


In [14]:
from scipy.stats import chi2_contingency

In [15]:
chi2_contingency(ct1)

# тест хи-квадрат показывает, что с вероятностью 99% наличие текста отзыва в отзывах на мужчин-фрилансеров и на женщин-фрилансерок значимо различается

(62.182501680642176,
 3.130573149307869e-15,
 1,
 array([[ 784.62396088, 5659.37603912],
        [ 460.37603912, 3320.62396088]]))

In [65]:
gendered.groupby('freelancer_gender').mean()

,texts_existence
freelancer_gender,
female,0.858628
male,0.911664


In [75]:
gendered[gendered.texts == 'Нет отзыва'].groupby('freelancer_gender')['marks'].value_counts()

freelancer_gender  marks        
female             Пять с плюсом    458
                   5                366
                   4                 75
                   3                  7
                   2                  3
                   1                  2
male               Пять с плюсом    158
                   5                129
                   4                 37
                   3                  4
                   1                  3
                   2                  3
Name: marks, dtype: int64

In [16]:
# почистим от текстов без отзывов

with_reviews = gendered[gendered.texts != 'Нет отзыва']

In [17]:
from nltk.corpus import stopwords
from pymystem3 import Mystem
from string import punctuation

In [18]:
mystem = Mystem()
russian_stopwords = stopwords.words("russian")

In [19]:
# базово предобработаем тексты

def preprocess_text(text):
    tokens = mystem.lemmatize(text.lower())
    tokens = [token for token in tokens if token not in russian_stopwords
              and token != " "
              and token.strip() not in punctuation]

    return tokens

In [20]:
with_reviews['texts_preprocessed'] = with_reviews.texts.apply(preprocess_text)

In [21]:
set_of_names = set(map(lambda x: x.lower(), female_names.T.values.tolist()[0])) | set(map(lambda x: x.lower(), male_names.T.values.tolist()[0]))

In [22]:
# проверим наличие имен в текстах отзывов
with_reviews['name_in_review'] = with_reviews.texts_preprocessed.apply(lambda text: len(set(text) & set_of_names) > 0)

In [23]:
ct2 = pd.crosstab(with_reviews['freelancer_gender'], with_reviews['name_in_review'])
ct2

name_in_review,False,True
freelancer_gender,,
female,3037,2496
male,2121,1326


In [24]:
chi2_contingency(ct2)

# тест хи-квадрат показывает, что с вероятностью 99% наличие имени в отзывах на мужчин-фрилансеров и на женщин-фрилансерок значимо различается

(38.06627389047619,
 6.838196863551224e-10,
 1,
 array([[3178.08619154, 2354.91380846],
        [1979.91380846, 1467.08619154]]))

проверить наличие отчества

In [83]:
with_reviews[with_reviews.texts.str.contains('вич ') | with_reviews.texts.str.contains('вна ')].drop_duplicates(subset='freelancer_name').freelancer_gender.value_counts()

female    34
male      20
Name: freelancer_gender, dtype: int64

In [94]:
with_reviews[with_reviews.texts.str.contains('вич ') | with_reviews.texts.str.contains('евна ') | with_reviews.texts.str.contains('евной') | with_reviews.texts.str.contains('овна ') | with_reviews.texts.str.contains('овной ')]

,marks,texts,author_names,profile,freelancer_name,freelancer_first_name,freelancer_gender,texts_existence,texts_preprocessed,name_in_review,gender_in_text,texts_joined,sentiment
515,6,Константин Васильевич очень доброжелательный и...,Катерина,https://profi.ru/it_freelance/designer/?seamle...,Константин Васильевич Тишин,Константин,male,True,"[васильевич, очень, доброжелательный, понимающ...",True,False,васильевич очень доброжелательный понимающий п...,1
775,6,Александра Юрьевна - замечательный педагог! За...,Елена,https://profi.ru/it_freelance/designer/?seamle...,Александра Юрьевна Мозжегорова,Александра,female,True,"[юрьевна, замечательный, педагог, занятие, нас...",True,False,юрьевна замечательный педагог занятие настольк...,1
808,6,Александра Юрьевна - замечательный педагог! За...,Елена,https://profi.ru/it_freelance/designer/?seamle...,Александра Юрьевна Мозжегорова,Александра,female,True,"[юрьевна, замечательный, педагог, занятие, нас...",True,False,юрьевна замечательный педагог занятие настольк...,1
1078,6,Владимир Сергеевич настроил мне рекламу в соц ...,Елена Попова,https://profi.ru/it_freelance/designer/?seamle...,Владимир Сергеевич Савочкин,Владимир,male,True,"[сергеевич, настраивать, реклама, соц, сеть, р...",True,False,сергеевич настраивать реклама соц сеть работа ...,1
1079,5,Выражаю свою благодарность Владимиру Сергеевич...,Андрей Кардаутов,https://profi.ru/it_freelance/designer/?seamle...,Владимир Сергеевич Савочкин,Владимир,male,True,"[выражать, свой, благодарность, сергеевич, сде...",True,False,выражать свой благодарность сергеевич сделать ...,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
10038,6,Необходимо было в короткие сроки освоить трасс...,Эдуард,https://profi.ru/it_freelance/designer/?seamle...,Анастасия Алексеевна Высочанская,Анастасия,female,True,"[необходимо, короткий, срок, осваивать, трасси...",True,False,необходимо короткий срок осваивать трассировка...,1
10266,5,Очень хороший специалист. Ольга Владиславовна ...,Ирина,https://profi.ru/it_freelance/designer/?seamle...,Ольга Владиславовна Лихварь,Ольга,female,True,"[очень, хороший, специалист, владиславовна, бы...",True,False,очень хороший специалист владиславовна быстро ...,1
10509,6,Евгения Александровна хороший специалист. Все ...,Тимофей,https://profi.ru/it_freelance/designer/?seamle...,Евгения Александровна Гафурова,Евгения,female,True,"[александровна, хороший, специалист, очень, по...",True,False,александровна хороший специалист очень понрави...,1
10510,6,Добрый день!С Евгенией Александровной Гафурово...,Юлия,https://profi.ru/it_freelance/designer/?seamle...,Евгения Александровна Гафурова,Евгения,female,True,"[добрый, день, александровна, гафурова, занима...",True,False,добрый день александровна гафурова заниматься ...,1


In [85]:
with_reviews.drop_duplicates(subset='freelancer_name').freelancer_gender.value_counts()

female    871
male      472
Name: freelancer_gender, dtype: int64

In [25]:
ct3 = pd.crosstab(with_reviews['marks'], with_reviews['name_in_review'])
ct3

name_in_review,False,True
marks,,
1,151,47
2,66,26
3,33,11
4,88,20
5,1690,1047
Пять с плюсом,3130,2671


In [26]:
chi2_contingency(ct3)

# имена в отзывах чаще встречаются в положительных отзывах

(117.0366358159197,
 1.3310762579059365e-23,
 5,
 array([[ 113.72873051,   84.27126949],
        [  52.84365256,   39.15634744],
        [  25.27305122,   18.72694878],
        [  62.03385301,   45.96614699],
        [1572.0986637 , 1164.9013363 ],
        [3332.022049  , 2468.977951  ]]))

In [27]:
# посмотрим на наличие слов, указывающих на гендер фрилансера в отзывах
gendered_words = ['девушка', 'женщина', 'мужчина', 'человек']

In [28]:
with_reviews['gender_in_text'] = with_reviews.texts_preprocessed.apply(lambda tokens: len(set(tokens) & set(gendered_words)) != 0)

In [29]:
ct4 = pd.crosstab(with_reviews['freelancer_gender'], with_reviews['gender_in_text'])
ct4

gender_in_text,False,True
freelancer_gender,,
female,5124,409
male,3247,200


In [30]:
chi2_contingency(ct4)

# с вероятностью 99% встречаемость слов, указывающих на гендер фрилансеров в текстах отзывов, различается

(8.24225694242785,
 0.004092621120725573,
 1,
 array([[5157.76648107,  375.23351893],
        [3213.23351893,  233.76648107]]))

In [31]:
# удалим имена из текстов отзывов

def del_name(tokens):
    return [token for token in tokens if token not in set_of_names]

In [32]:
with_reviews['texts_preprocessed'] = with_reviews['texts_preprocessed'].apply(del_name)

In [34]:
with_reviews.loc[with_reviews.marks == 'Пять с плюсом', 'marks'] = 6

In [35]:
with_reviews['marks'] = with_reviews.marks.astype(int)

In [71]:
with_reviews.groupby('freelancer_gender')['marks'].mean()

freelancer_gender
female    5.514007
male      5.491152
Name: marks, dtype: float64

Распределения оценок не различаются

In [36]:
with_reviews['texts_joined'] = with_reviews.texts_preprocessed.apply(lambda tokens: " ".join(tokens))

In [37]:
women = with_reviews[with_reviews.freelancer_gender == 'female'][['texts_joined', 'marks']]
men = with_reviews[with_reviews.freelancer_gender == 'male'][['texts_joined', 'marks']]

In [38]:
x_train_m, x_test_m, y_train_m, y_test_m = train_test_split(men.texts_joined, men.marks, random_state=42)
x_train_f, x_test_f, y_train_f, y_test_f = train_test_split(women.texts_joined, women.marks, random_state=42)

In [39]:
vec_m = TfidfVectorizer(ngram_range=(1, 3))
vec_train_m = vec_m.fit_transform(x_train_m)
vec_test_m = vec_m.transform(x_test_m)


scaler_m = MaxAbsScaler()
vec_train_m = scaler_m.fit_transform(vec_train_m)
vec_test_m = scaler_m.transform(vec_test_m)

In [40]:
linear_m = LinearRegression()
linear_m.fit(vec_train_m, y_train_m)
preds_m = linear_m.predict(vec_test_m)

In [41]:
mean_squared_error(preds_m, y_test_m)

0.3667394686425137

In [42]:
eli5.sklearn.explain_weights_sklearn(linear_m, vec=vec_m)

Explanation(estimator='LinearRegression()', description="\nFeatures with largest coefficients.\nCaveats:\n1. Be careful with features which are not\n   independent - weights don't show their importance.\n2. If scale of input features is different then scale of coefficients\n   will also be different, making direct comparison between coefficient values\n   incorrect.\n3. Depending on regularization, rare features sometimes may have high\n   coefficients; this doesn't mean they contribute much to the\n   classification result for most examples.\n", error=None, method='linear model', is_regression=True, targets=[TargetExplanation(target='y', feature_weights=FeatureWeights(pos=[FeatureWeight(feature='<BIAS>', weight=5.211428833490206, std=None, value=None), FeatureWeight(feature='хороший рекомендовать', weight=1.202299445195538, std=None, value=None), FeatureWeight(feature='спасибо рекомендовать', weight=1.04953809837673, std=None, value=None), FeatureWeight(feature='быстро качественно выполнять', weight=0.9826203631114058, std=None, value=None), FeatureWeight(feature='отлично рекомендовать', weight=0.9372800754096071, std=None, value=None)], neg=[FeatureWeight(feature='пропадать', weight=-1.9299199860408478, std=None, value=None), FeatureWeight(feature='никак', weight=-1.8610131368445082, std=None, value=None), FeatureWeight(feature='безответственный', weight=-1.5907104302592272, std=None, value=None), FeatureWeight(feature='безответственный человек', weight=-1.560092030129058, std=None, value=None), FeatureWeight(feature='ничто', weight=-1.3760602186388347, std=None, value=None), FeatureWeight(feature='недобросовестный специалист', weight=-1.3737325567317764, std=None, value=None), FeatureWeight(feature='недобросовестный', weight=-1.3737325567317764, std=None, value=None), FeatureWeight(feature='недобросовестный специалист рекомендовать', weight=-1.3737325567317764, std=None, value=None), FeatureWeight(feature='никак сделать', weight=-1.302575217396673, std=None, value=None), FeatureWeight(feature='консультация', weight=-1.2114291190992899, std=None, value=None), FeatureWeight(feature='рекомендовать', weight=-1.211427547160628, std=None, value=None), FeatureWeight(feature='человек', weight=-1.0606261694620227, std=None, value=None), FeatureWeight(feature='предоплата', weight=-0.9566769216279711, std=None, value=None), FeatureWeight(feature='относиться работа', weight=-0.8307066240826708, std=None, value=None), FeatureWeight(feature='четко срок', weight=-0.827205948947997, std=None, value=None)], pos_remaining=29481, neg_remaining=17563), proba=None, score=None, weighted_spans=None, heatmap=None)], feature_importances=None, decision_tree=None, highlight_spaces=None, transition_features=None, image=None)

In [44]:
vec_f = TfidfVectorizer(ngram_range=(1, 3))
vec_train_f = vec_f.fit_transform(x_train_f)
vec_test_f = vec_f.transform(x_test_f)


scaler_f = MaxAbsScaler()
vec_train_f = scaler_f.fit_transform(vec_train_f)
vec_test_f = scaler_f.transform(vec_test_f)

In [45]:
linear_f = LinearRegression()
linear_f.fit(vec_train_f, y_train_f)
preds_f = linear_f.predict(vec_test_f)

In [46]:
mean_squared_error(preds_f, y_test_f)

0.6104155488984659

In [47]:
eli5.sklearn.explain_weights_sklearn(linear_f, vec=vec_f)

Explanation(estimator='LinearRegression()', description="\nFeatures with largest coefficients.\nCaveats:\n1. Be careful with features which are not\n   independent - weights don't show their importance.\n2. If scale of input features is different then scale of coefficients\n   will also be different, making direct comparison between coefficient values\n   incorrect.\n3. Depending on regularization, rare features sometimes may have high\n   coefficients; this doesn't mean they contribute much to the\n   classification result for most examples.\n", error=None, method='linear model', is_regression=True, targets=[TargetExplanation(target='y', feature_weights=FeatureWeights(pos=[FeatureWeight(feature='<BIAS>', weight=4.824011185902603, std=None, value=None), FeatureWeight(feature='молодец', weight=1.1759892832352974, std=None, value=None), FeatureWeight(feature='шикарно', weight=1.1759891530867288, std=None, value=None), FeatureWeight(feature='волшебница', weight=1.1759891474989652, std=None, value=None), FeatureWeight(feature='бал', weight=1.1759891455731388, std=None, value=None), FeatureWeight(feature='четко', weight=1.1759890696761741, std=None, value=None), FeatureWeight(feature='талант', weight=1.1759888856062497, std=None, value=None), FeatureWeight(feature='проходить', weight=1.1759888736803141, std=None, value=None), FeatureWeight(feature='очень', weight=1.175988869627603, std=None, value=None)], neg=[FeatureWeight(feature='лохотрон', weight=-3.824011357589294, std=None, value=None), FeatureWeight(feature='пунктуальность', weight=-2.8240114474609594, std=None, value=None), FeatureWeight(feature='оценка', weight=-2.8240113241428997, std=None, value=None), FeatureWeight(feature='аккуратно', weight=-2.8240111153817997, std=None, value=None), FeatureWeight(feature='денисовна супер', weight=-2.068208764790725, std=None, value=None), FeatureWeight(feature='денисовна', weight=-2.056343041563108, std=None, value=None), FeatureWeight(feature='профессиональный подход', weight=-1.5163557823332316, std=None, value=None), FeatureWeight(feature='некомпетентный', weight=-1.472310444310844, std=None, value=None), FeatureWeight(feature='некомпетентный специалист', weight=-1.4675478734528429, std=None, value=None), FeatureWeight(feature='профессиональный', weight=-1.3492108981748967, std=None, value=None), FeatureWeight(feature='логотип понравиться', weight=-1.3356315636846954, std=None, value=None)], pos_remaining=47661, neg_remaining=28983), proba=None, score=None, weighted_spans=None, heatmap=None)], feature_importances=None, decision_tree=None, highlight_spaces=None, transition_features=None, image=None)

In [48]:
# построим бинарную логистическую регрессию, где объединим оценки 5 и 5+ в первый класс и 1-4 во второй класс

with_reviews['sentiment'] = 1
with_reviews.loc[with_reviews.marks.isin([1, 2, 3, 4]), 'sentiment'] = 0

In [49]:
with_reviews.sentiment.value_counts()

1    8538
0     442
Name: sentiment, dtype: int64

In [169]:
def has_cyrillic(text):
    """
    Проверяет, написано ли слово на кириллице
    """
    return bool(re.search('[а-яА-Я]', text))

In [190]:
def clean(text_tokenized):
    """
    Функция для выделения прилагательных в текстах отзывов (включает чистку от слов на латинице)
    """
    tokens = [token for token in text_tokenized if token.isalpha() and (token != ' ') and has_cyrillic(token) is True]
    tokens_res = []
    for token in tokens:
        try:
            if mystem.analyze(token)[0]['analysis'][0]['gr'].startswith('A='):
                tokens_res.append(token)
        except:
            pass
    return " ".join(tokens_res)

In [191]:
with_reviews['adjectives'] = with_reviews.texts_preprocessed.apply(clean)

In [192]:
women_bin = with_reviews[with_reviews.freelancer_gender == 'female'][['texts_joined', 'sentiment', 'adjectives']]
men_bin = with_reviews[with_reviews.freelancer_gender == 'male'][['texts_joined', 'sentiment', 'adjectives']]

In [193]:
cv_f = CountVectorizer()
cv_fit_f = cv_f.fit_transform(women_bin.adjectives.values)
freqs_f = dict(zip(cv_f.get_feature_names(), np.asarray(cv_fit_f.sum(axis=0))[0]))

In [195]:
cv_m = CountVectorizer()
cv_fit_m = cv_m.fit_transform(men_bin.adjectives.values)
freqs_m = dict(zip(cv_m.get_feature_names(), np.asarray(cv_fit_m.sum(axis=0))[0]))

In [197]:
women_frame = pd.Series(dict(sorted(freqs_f.items(), key=lambda item: item[1], reverse=True))).to_frame().reset_index().rename(columns={0: 'female_freq', 'index': 'adjective'})
men_frame = pd.Series(dict(sorted(freqs_m.items(), key=lambda item: item[1], reverse=True))).to_frame().reset_index().rename(columns={0: 'male_freq', 'index': 'adjective'})

In [199]:
adjectives = women_frame.merge(men_frame, on='adjective', how='outer')
adjectives['share_f'] = adjectives.female_freq / women_bin.shape[0]
adjectives['share_m'] = adjectives.male_freq / men_bin.shape[0]
adjectives

,adjective,female_freq,male_freq,share_f,share_m
0,довольный,729.0,419.0,0.131755,0.121555
1,отличный,614.0,402.0,0.110971,0.116623
2,большой,599.0,317.0,0.108260,0.091964
3,хороший,529.0,314.0,0.095608,0.091094
4,огромный,329.0,219.0,0.059461,0.063534
...,...,...,...,...,...
1134,цитрусовый,NaN,1.0,NaN,0.000290
1135,чистоплотный,NaN,1.0,NaN,0.000290
1136,штатный,NaN,1.0,NaN,0.000290
1137,юморный,NaN,1.0,NaN,0.000290


По-моему очень интересно то, насколько совпадает доля среди нейтральных слов (топ-5). Прекрасный, приятный, замечательный, красивый, отзывчивый - доля сильно выше в женских отзывах. Грамотный, настоящий (специалист), аккуратный - среди мужчин.

In [51]:
x_train_mb, x_test_mb, y_train_mb, y_test_mb = train_test_split(men_bin.texts_joined, men_bin.sentiment, random_state=42)
x_train_fb, x_test_fb, y_train_fb, y_test_fb = train_test_split(women_bin.texts_joined, women_bin.sentiment, random_state=42)

In [52]:
vec_mb = TfidfVectorizer(ngram_range=(1, 3), max_features=10000)
vec_train_mb = vec_mb.fit_transform(x_train_mb)
vec_test_mb = vec_mb.transform(x_test_mb)


scaler_mb = MaxAbsScaler()
vec_train_mb = scaler_mb.fit_transform(vec_train_mb)
vec_test_mb = scaler_mb.transform(vec_test_mb)

In [53]:
grid = {"C": np.logspace(-3, 3, 7),
        "penalty": ["l1", "l2", None],
        "solver": ['lbfgs', 'liblinear']} # l1 lasso l2 ridge
logreg = LogisticRegression()
logreg_cv = GridSearchCV(logreg, grid, cv=5)
logreg_cv.fit(vec_train_mb, y_train_mb)

GridSearchCV(cv=5, estimator=LogisticRegression(),
             param_grid={'C': array([1.e-03, 1.e-02, 1.e-01, 1.e+00, 1.e+01, 1.e+02, 1.e+03]),
                         'penalty': ['l1', 'l2', None],
                         'solver': ['lbfgs', 'liblinear']})

In [54]:
logreg_cv.best_params_

{'C': 1000.0, 'penalty': 'l2', 'solver': 'lbfgs'}

In [55]:
log_m = LogisticRegression(C=1000.0, penalty='l2', solver='lbfgs')
log_m.fit(vec_train_mb, y_train_mb)
preds_mb = log_m.predict(vec_test_mb)

In [56]:
print(classification_report(preds_mb, y_test_mb))

              precision    recall  f1-score   support

           0       0.70      0.94      0.80        35
           1       1.00      0.98      0.99       827

    accuracy                           0.98       862
   macro avg       0.85      0.96      0.90       862
weighted avg       0.99      0.98      0.98       862



In [57]:
eli5.sklearn.explain_weights_sklearn(log_m, vec=vec_mb, top=100)

Weight?,Feature
+10.550,очень
+8.048,молодец
+7.835,быстро
+7.698,отлично
+7.170,отличный
+6.909,качественно
+6.490,спасибо
+5.543,оперативно
+5.289,хороший специалист
+5.247,понравиться


In [58]:
vec_fb = TfidfVectorizer(ngram_range=(1, 3), max_features=10000)
vec_train_fb = vec_fb.fit_transform(x_train_fb)
vec_test_fb = vec_fb.transform(x_test_fb)


scaler_fb = MaxAbsScaler()
vec_train_fb = scaler_fb.fit_transform(vec_train_fb)
vec_test_fb = scaler_fb.transform(vec_test_fb)

In [59]:
logreg_f = LogisticRegression()
logreg_cv_f = GridSearchCV(logreg_f, grid, cv=5)
logreg_cv_f.fit(vec_train_fb, y_train_fb)

GridSearchCV(cv=5, estimator=LogisticRegression(),
             param_grid={'C': array([1.e-03, 1.e-02, 1.e-01, 1.e+00, 1.e+01, 1.e+02, 1.e+03]),
                         'penalty': ['l1', 'l2', None],
                         'solver': ['lbfgs', 'liblinear']})

In [60]:
logreg_cv_f.best_params_

{'C': 10.0, 'penalty': 'l2', 'solver': 'lbfgs'}

In [61]:
log_f = LogisticRegression(C=10, penalty='l2', solver='lbfgs')
log_f.fit(vec_train_fb, y_train_fb)
preds_fb = log_f.predict(vec_test_fb)

In [62]:
print(classification_report(preds_fb, y_test_fb))

              precision    recall  f1-score   support

           0       0.63      0.98      0.77        52
           1       1.00      0.98      0.99      1332

    accuracy                           0.98      1384
   macro avg       0.81      0.98      0.88      1384
weighted avg       0.99      0.98      0.98      1384



In [63]:
eli5.sklearn.explain_weights_sklearn(log_f, vec=vec_fb, top=100)

Weight?,Feature
+3.945,спасибо
+3.503,очень
+3.475,качественно
+3.246,отлично
+3.196,рекомендовать
+3.087,отличный
+2.694,<BIAS>
+2.668,работа
+2.421,весь
+2.322,оперативно


в топе у женщин вообще нет слов профессионал, компетентный и т. д.
зато они есть в антитопе, а вот у мужчин наоборот